
# Original Glicko

The original Glicko system was created by Mark Glickman in 1995. You can read about it in detail on Mark Glickman's website http://glicko.net/. It was one of, if not the first attempts at incorporating uncertainty into a Elo-style ratings system. It often outperforms basic Elo for that reason, while keeping most of Elo's advantages. There are still very few parameters to tune, and it is still fairly universal, and it is computationally quick. While Glicko is generally outperformed by some more computationally intensive and new methods, it serves as a great baseline while rating player or team performance. 

Glicko 2 was developed later, and has an additional parameter. However, I haven't found much difference between the two.

### Advantages

- Quicker to converge than Elo
- Better at updating once converged than Elo
- Simplicity
- Computational speed
- Universality
- Good at handling cross-league, cross-conference play
- Not much tuning needed - the default RD of 350 usually gives you reasonable results, rarely is a much different value needed

### Disadvantages

- Cannot account for score difference. A win by 50 is the same as a win by 5.
- There is no attempt to model style of play or player idiosyncracies
- Without some edits, it will not put more weight on certain games than others, i.e., if a team or player is particularly motivated. 
- As with any rating system, if players are allowed to selectively choose opponents, it can ruin the fidelity of the ratings
- Is meant for 1v1 ratings, or team v team. Cannot innately handle 1vMany or Many vs. Many.



In [2]:

import gc
import os

import numpy as np
import pandas as pd

from copy import copy
from tqdm import tqdm
from oddsmaker.state_space import Glicko
from pandas.api.types import is_datetime64_any_dtype as is_datetime


ImportError: cannot import name 'Glicko' from 'oddsmaker.state_space' (C:\Users\Blake\anaconda3\lib\site-packages\oddsmaker\state_space.py)


## Quick Start

Here, I'll show how you can use the oddsmaker Glicko class to model team ratings, and compare it to Elo. The class takes data in a specific format. Once you learn that format, it becomes powerful.


In [8]:

test = np.random.random((4,3,2))

test[0]


array([[0.8397777 , 0.61403004],
       [0.14504105, 0.89587469],
       [0.42231017, 0.60442336]])

In [141]:

class Glicko():

    """
    
    Glicko rating system implementation

    Parameters
    ----------
    data : pandas dataframe
        Dataframe containing the following columns:
            - protag_id: protagonist id (player or team)
            - antag_id: antagonist id (player or team)
            - stat: Specifies stat involved
            - result: 0 for loss, 1 for win, 0.5 for tie
            - is_home (optional): 1 for home, -1 for away, 0 for neutral
            - date or rating period: date of game or rating period of game
    protag_id : str, optional
        Name of protagonist id column, by default 'team_name'
    antag_id : str, optional    
        Name of antagonist id column, by default 'opp_name' 
    result_col : str, optional
        Name of result column, by default 'result'
    RD : int, optional
        Initial RD (sigma, or standard deviation) value, by default 350
    max_RD : int, optional    
        Maximum RD value, by default 350. Usually same as initial RD
    time_param : float, optional
        Time parameter, by default 1. Multiplies by time elapsed. The lower the value, the slower the rating variance goes back to default
    hfa : int, float, or dict, optional
        Home field advantage value(s) to use for each stat. Used to create 'hfa' column. If 'hfa' column already provided, then ignores. If int or float, applies to all stats. If dict, applies to each stat individually. If None, no home field advantage is used. Default is None.
    priors : dict, optional
        Dictionary of priors to use for each stat. If None, no priors are used. Default is None.


    """

    def __init__(
            self, 
            data, 
            protag_id='protag', 
            antag_id='antag', 
            result_col='result', 
            RD=350, 
            max_RD=350, 
            time_param=1, 
            matchup_type='1v1',
            hfa=None, 
            priors=None
        ):

        self.data = data.copy()
        self.protag_id = protag_id
        self.antag_id = antag_id
        self.result_col = result_col
        self.RD = RD
        self.max_RD = max_RD
        self.time_param = time_param
        self.hfa = hfa
        self.matchup_type = matchup_type
        self.col_names = list(self.data)
        self.col_names = [cn.lower().strip() for cn in self.col_names]
        self.data.columns=self.col_names
        
        ## constant
        self.q = np.log(10)/400

        ### validation checks
        assert(self.protag_id in list(data)), f"{self.protag_id} not in columns, please specify team column name with protag_id argument"
        assert(self.antag_id in list(data)), f"{self.antag_id} not in columns, please specify opponent column name with antag_id argument"

        assert('stat' in list(self.data)), 'No stat column, please add a stat name column to your data'
        self.stats = sorted(list(self.data.stat.unique()))
        self.result_col = result_col
        assert(self.result_col in list(self.data)), 'Please include an outcome/result column, can specify the name with result col argument'
        assert((('date' in self.col_names)|('rating_period' in self.col_names))), "Need either a date column or a rating period column"
        if 'date' in self.col_names:
            self.period_type = 'date'
            assert(is_datetime(self.data[self.period_type].dtype)), "Date column must be of type datetime"
        else:
            self.period_type = 'rating_period'

        assert(self.matchup_type in ['1v1','1vMany','ManyvMany','Multiple']), "matchup_type must be one of ['1v1','1vMany','ManyvMany','Multiple']"
        if self.matchup_type == 'Multiple':
            assert('matchup_type' in list(self.data)), "matchup_type is set to 'Multiple', but no matchup_type column found in data"
            for mtype in self.data.matchup_type.unique():
                assert(mtype in ['1v1','1vMany','ManyvMany']), "Unrecognized matchup_type, must be one of ['1v1','1vMany','ManyvMany']"
        else:   
            self.data['matchup_type'] = self.matchup_type

        self.num_protags = len(self.data[self.protag_id].unique())
        self.num_antags = len(self.data[self.antag_id].unique())
        self.num_stats = len(self.data.stat.unique())
        self.num_games = len(self.data)//2

        self.rating_matrix = np.ones((self.num_protags, self.num_stats, 3))*1500
        self.rating_matrix[:,:,1] = self.RD
        self.rating_matrix[:,:,2] = 120 # default time off. Not used initially, so just placeholder

        ### check if data is symmetrical, i.e., for every team a vs team b there is a team b vs team a
        protags = self.data.groupby(['rating_period','stat'])[self.protag_id].apply(set).reset_index().copy()
        antags =self.data.groupby(['rating_period','stat'])[self.antag_id].apply(set).reset_index().copy()
        sym_test = protags.merge(antags, how='left', on=['rating_period','stat'])
        sym_test['sym_diff'] = sym_test[[self.protag_id,self.antag_id]].apply(lambda x: len(x[self.protag_id].symmetric_difference(x[self.antag_id])), axis=1)
        sym_val = sym_test['sym_diff'].mean()
    
        if sym_val > 0.05:
            print("Warning: data is not symmetrical. There should be two rows per match for this class")
            raise ValueError(" At least {}% of games are missing.".format(round(100*sym_val,2)))
        elif sym_val > 0:
            print("Warning: a few games are likely missing their symmetrical partner.")

        ### index maps
        self.protag2index = {}
        for i,protag_id in enumerate(self.data[self.protag_id].unique()):
            self.protag2index[protag_id] = i
            
        self.stat2index = {}
        for j,stat in enumerate(self.stats):
            self.stat2index[stat] = j
            
        if priors is not None:
            self.priors = priors
            assert(type(self.priors)==dict), "priors must be a dictionary"
            self._implement_priors()

        self._add_hfa()
            
        ### initialize ratings
        self.data['protag_idx'] = self.data[self.protag_id].copy().map(self.protag2index)
        self.data['antag_idx'] = self.data[self.antag_id].copy().map(self.protag2index)
        self.data['stat_idx'] = self.data['stat'].copy().map(self.stat2index)


    def _implement_priors(self):

        """
        Implements priors if they are provided
        """

        for protag_id, stat_dict in self.priors.items():
            assert(type(stat_dict)==dict), "Each protag_id key in priors dict must be a dict of stat:rating pairs"
            for stat, rating_dict in stat_dict.items():
                assert(stat in self.stats), f"{stat} is not a stat in the dataset"
                assert('rating' in rating_dict), "Each stat key in priors dict must have a 'rating' key"
                assert('RD' in rating_dict), "Each stat key in priors dict must have a 'RD' key"
                self.rating_matrix[self.protag2index[protag_id], self.stat2index[stat], 0] = rating_dict['rating']
                self.rating_matrix[self.protag2index[protag_id], self.stat2index[stat], 1] = rating_dict['RD']
                if 'time_off' in rating_dict:
                    self.rating_matrix[self.protag2index[protag_id], self.stat2index[stat], 2] = rating_dict['time_off']

        return

    def _add_hfa(self, new_data=False):

        """
        Adds home field advantage to data if it is provided
        """

        if self.hfa is None:
            self.has_hfa=False
            return
    
        if ((new_data == True)&('hfa' not in list(self.data))):
            raise ValueError("hfa provided for original data, but no hfa column found in new data. Please add hfa column similar to previously provided data")

        if type(self.hfa)==dict:
            self.data['hfa'] = self.data['stat'].map(self.hfa).copy()
        elif type(self.hfa) in [int, float]:
            self.data['hfa'] = self.hfa
        else:
            raise ValueError("hfa must be a dictionary, float, or an integer")
            
        self.has_hfa=True

        return

    def _d_sq_calc(self, x):
        return 1/(self.q**2 * (x.g_opps**2*(x.exp_res)*(1-x.exp_res)).sum())

    def _mu_adj_calc(self, x):
        return (x.g_opps*(x.result-x.exp_res)).sum()

    def _rating_period_update(self, pregame_protag_ratings, pregame_antag_ratings, rp):

        """
        Updates ratings for a single rating period
        """

        protag_mus = pregame_protag_ratings[:,0]
        protag_RDs = pregame_protag_ratings[:,1]

        antag_mus = pregame_antag_ratings[:,0]
        antag_RDs = pregame_antag_ratings[:,1]

        g_opps = 1/np.sqrt(1+(3*self.q**2*antag_RDs**2)/np.pi**2)
        exp_res = 1/(1+10**(-g_opps*(protag_mus-antag_mus)/400))

        pids = rp['protag_idx'].values
        aids = rp['antag_idx'].values
        sids = rp['stat_idx'].values

        temp_df = pd.DataFrame({
            'pid':pids, 
            'stat':sids,
            'mus':protag_mus,
            'rds':protag_RDs,
            'g_opps':g_opps,
            'exp_res':exp_res,
            'result':rp[self.result_col].values
        })

        d_sq = temp_df.groupby(['pid','stat']).apply(lambda x: self._d_sq_calc(x)).reset_index()
        ids = d_sq[['pid','stat']]
        d_sq = d_sq[0].values

        # np.sum([g_opps[i]*(results[i]-exp_results[i]) for i in range(self.num_opps)])
        mu_adj_term = temp_df.groupby(['pid','stat']).apply(lambda x: self._mu_adj_calc(x)).reset_index()
        mu_adj_term = mu_adj_term[0].values
        
        print(protag_mus)
        print(protag_RDs)
        print(mu_adj_term)
        single_data = temp_df.drop_duplicates(subset=['pid','stat'])[['pid','stat','mus','rds']]
        single_data = ids.merge(single_data, how='left', on=['pid','stat'])
        single_protag_mus = single_data['mus'].values
        single_protag_RDs = single_data['rds'].values

        new_protag_mus = single_protag_mus + self.q/(1/(single_protag_RDs**2) + 1/d_sq)*mu_adj_term
        new_protag_RDs = np.sqrt(1/(1/(single_protag_RDs**2) + 1/d_sq))

        postgame_protag_ratings = np.zeros(ids.shape)
        postgame_protag_ratings[:,0] = new_protag_mus
        postgame_protag_ratings[:,1] = new_protag_RDs

        return postgame_protag_ratings

    def create_pregame_ratings(self):

        """
        Runs through all games in history and creates pregame ratings
        """

        self.data = self.data.sort_values(by=[self.period_type,'stat']).reset_index(drop=True)

        quick_iterator = self.data.groupby([self.period_type,'stat'])
        for rp_index, rating_period in tqdm(quick_iterator, total=len(quick_iterator)):

            pregame_protag_ratings = self.rating_matrix[rating_period.protag_idx.values, rating_period.stat_idx.values, :]
            pregame_antag_ratings = self.rating_matrix[rating_period.antag_idx.values, rating_period.stat_idx.values, :]

            if self.has_hfa:
                pregame_protag_ratings = pregame_protag_ratings+((rating_period.hfa.values*rating_period.is_home.values)/2)
                pregame_antag_ratings = pregame_antag_ratings-((rating_period.hfa.values*rating_period.is_home.values)/2)

            postgame_protag_ratings = self._rating_period_update(pregame_protag_ratings, pregame_antag_ratings, rating_period.copy())

            if self.has_hfa:
                postgame_protag_ratings = pregame_protag_ratings-((rating_period.hfa.values*rating_period.is_home.values)/2)
            print(postgame_protag_ratings)

        return


In [142]:


def make_symmetrical(df, protag_id='protag', antag_id='antag'):
    
    antag_df = df.copy()
    antag_df['result'] = 1-antag_df['result'].copy()
    antag_df = antag_df.rename(columns={protag_id:antag_id, antag_id:protag_id})
    
    df = pd.concat([df, antag_df], axis=0).reset_index(drop=True)
    
    return df

data = pd.DataFrame({
    'rating_period':[1,1,1],
    'protag':[1,1,1],
    'antag':[2,3,4],
    'stat':np.repeat('strokes_gained',3),
    'result':[1,0,0]
})

data = make_symmetrical(data)

priors = {
    1:{'strokes_gained':{'rating':1500, 'RD':200}},
    2:{'strokes_gained':{'rating':1400, 'RD':30}},
    3:{'strokes_gained':{'rating':1550, 'RD':100}},
    4:{'strokes_gained':{'rating':1700, 'RD':300}}
}

glicko = Glicko(data, priors=priors)
glicko.create_pregame_ratings()



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 166.67it/s]

[1500. 1500. 1500. 1400. 1550. 1700.]
[200. 200. 200.  30. 100. 300.]
[-0.27202894 -0.32153156  0.37110077  0.23173759]
[[1464.10646276  151.39890245]
 [1398.34251247   29.92509104]
 [1570.18760945   97.21172957]
 [1784.35028135  251.45899758]]


In [71]:

test_protags = np.array([
    [[1500, 200], [1600, 200]],
    [[1500, 200], [1500, 200]],
    [[1500, 200], [1400, 200]],
    [[1400, 200], [1600, 200]],
    [[1400, 200], [1500, 200]],
    [[1400, 200], [1400, 200]],
    [[1500, 150], [1600, 200]],
    [[1500, 150], [1500, 200]],
    [[1500, 150], [1400, 200]],
])

test_antags = np.array([
    [[1400, 30], [1500, 200]],
    [[1550, 100], [1500, 200]],
    [[1700, 300], [1500, 200]],
    [[1400, 30], [1500, 200]],
    [[1550, 100], [1500, 200]],
    [[1700, 300], [1500, 200]],
    [[1400, 30], [1500, 200]],
    [[1550, 100], [1500, 200]],
    [[1700, 300], [1500, 200]],
])

test_protags.shape

protag_mus = test_protags[:,:,0]
protag_rds = test_protags[:,:,1]
antag_mus = test_antags[:,:,0]
antag_rds = test_antags[:,:,1]


q=np.log(10)/400
g_opps = 1/np.sqrt((1+(3*q**2)*(antag_rds**2)/np.pi**2))
exp_res = 1/(1+10**((-g_opps*(protag_mus-antag_mus))/400))
exp_res

# len(stats)
# stats
#d_sq = 1/(q**2



array([[0.63946774, 0.61916545],
       [0.43184235, 0.5       ],
       [0.30284073, 0.38083455],
       [0.5       , 0.61916545],
       [0.30512404, 0.5       ],
       [0.2225772 , 0.38083455],
       [0.63946774, 0.61916545],
       [0.43184235, 0.5       ],
       [0.30284073, 0.38083455]])

In [ ]:

d_sq



In [90]:
pids = np.repeat(np.array([1,1,1,2,2,2,3,3,3]),2)
stats = list(np.tile(['driving','irons'],9))
temp_df = pd.DataFrame({
    'mu':protag_mus.reshape(-1),
    'rd':protag_mus.reshape(-1),
    'g_opps':g_opps.reshape(-1),
    'exp_res':exp_res.reshape(-1),
    'pid':pids,
    'stat':stats
})
temp_df


,mu,rd,g_opps,exp_res,pid,stat
0,1500,1500,0.995498,0.639468,1,driving
1,1600,1600,0.844281,0.619165,1,irons
2,1500,1500,0.953149,0.431842,1,driving
3,1500,1500,0.844281,0.500000,1,irons
4,1500,1500,0.724235,0.302841,1,driving
5,1400,1400,0.844281,0.380835,1,irons
6,1400,1400,0.995498,0.500000,2,driving
7,1600,1600,0.844281,0.619165,2,irons
8,1400,1400,0.953149,0.305124,2,driving
9,1500,1500,0.844281,0.500000,2,irons


In [83]:

def d_sq_calc(x):
    return 1/(q**2 * (x.g_opps**2*(x.exp_res)*(1-x.exp_res)).sum())
d_sq = temp_df.groupby(['pid','stat']).apply(lambda x: d_sq_calc(x)).reset_index()
ids = d_sq[['pid','stat']].values
d_sq = d_sq[0].values


array([53685.74290198, 58670.26196183, 56817.55104186, 58670.26196183,
       53685.74290198, 58670.26196183])

In [49]:

def d_sq_calc(x):
    return 1/(q**2 * (x.g_opps**2*(x.exp_res)*(1-x.exp_res)).sum())

d_sq = temp_df.groupby('pid').apply(lambda x: d_sq_calc(x))
protag_ids = d_sq.index.values
d_sq = d_sq.values


# self.player_a.mu = self.player_a.mu + (self.q/(1/self.player_a.rd**2+1/d_sq)*np.sum([g_opps[i]*(results[i]-exp_results[i]) for i in range(self.num_opps)]))
# self.player_a.rd = np.sqrt(1/(1/self.player_a.rd**2 + 1/d_sq))



In [88]:
d_sq.shape

(6,)

In [86]:
### may have to drop duplicates to get these but possible

pmu_1d = np.array([1500, 1600, ])
prd_1d = protag_rds.reshape(-1,1)
# new_pmu = pmu_1d + (q/(1/))
prd_1

array([[200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [150],
       [200],
       [150],
       [200],
       [150],
       [200]])

In [50]:
protag_ids

array([1, 2, 3], dtype=int64)

In [17]:

protag_mus = np.random.random((4,2))*400+1300


protag_mus


array([[1599.57187889, 1581.564999  ],
       [1680.72617651, 1332.23662836],
       [1688.09097059, 1611.23380995],
       [1592.79405142, 1485.79695658]])

In [18]:

protag_rds = np.random.random((4,2))*100+250


In [19]:

antag_mus = np.random.random((4,2))*400+1300
antag_mus


array([[1492.28885754, 1571.34876168],
       [1691.84871914, 1696.61314767],
       [1529.69218904, 1663.54154893],
       [1322.19870591, 1505.5040629 ]])

In [21]:

antag_rds = np.random.random((4,2))*100+250
antag_rds


array([[261.01313845, 288.59590174],
       [300.86893221, 325.89439742],
       [318.84359453, 332.82385098],
       [316.50336572, 269.32448527]])

In [26]:
q=np.log(10)/400
g_opps = 1/np.sqrt((1+(3*q**2)*(antag_rds**2)/np.pi**2))
exp_res = 1/(1+10**((-g_opps*(protag_mus-antag_mus))/400))
# g_opps.append(g_opp)


In [29]:
results = np.round(np.random.random((4,2)))
results = results.astype(int)
results

array([[0, 1],
       [1, 1],
       [0, 1],
       [1, 1]])

In [27]:

# new_mus = protag_mus + (q/(1/protag_rds**2+1/d_sq)*np.sum([g_opps[i]*(results[i]-exp_results[i]) for i in range(self.num_opps)]))
# new_rds = np.sqrt(1/(1/protag_rds**2 + 1/d_sq))


array([[0.61670547, 0.51084024],
       [0.48842546, 0.18877934],
       [0.65496187, 0.44843158],
       [0.75006736, 0.47845482]])